In [17]:
import geohash
import datetime

def parseLine(line):
    variables = line.split("\t")
    try:
        milliseconds = int(variables[0])
        dt = datetime.datetime.fromtimestamp(milliseconds/1000.0)

        lat = float(variables[1])
        lon = float(variables[2])
        temperature = float(variables[10]) - 273.0
        precipitation = float(variables[13])
        gh = geohash.encode(lat, lon)
        return (gh, dt.month, temperature, precipitation)
    except:
        return ('', 0, 0, 0)

prefix = "9p"
    
#text_file = sc.textFile("hdfs://orion11:12001/pj3/3hr_sample/sampled_2015/part-00004-300232b0-d488-4017-8741-efda0762a4d3-c000.tdv.gz")
text_file = sc.textFile("hdfs://orion11:12001/pj3/3hr_sample/sampled_2015/")

# (GeoHash, month, temperature, precipitation)
parsed_data = text_file \
    .map(lambda line: parseLine(line)) \
    .filter(lambda data: data[0].startswith("9p")) \


In [32]:
# average temperature
month_temperature_pair = parsed_data.map(lambda data: (data[1], data[2]))

allTemps = month_temperature_pair.groupByKey()

for element in allTemps.collect():
    count = 0
    temp_sum = 0
    for temp in element[1]:
        count += 1
        temp_sum += temp
    print((element[0], temp_sum/count))



(1, 13.024943762371255)
(2, 13.063885973780156)
(3, 12.995700196384332)
(4, 12.994589544131415)
(5, 13.72163984510928)
(6, 15.27776004023778)
(7, 17.534185501372868)
(8, 18.66300041487601)
(9, 18.471258106045905)
(10, 17.700511886549062)
(11, 15.804166341370385)
(12, 13.736084409453184)


In [30]:
# maximum temperature

maxTemps = month_temperature_pair.reduceByKey(lambda a, b: max(a,b))
for element in maxTemps.collect():
    print(element)


(1, 23.96746999999999)
(2, 25.136439999999993)
(3, 30.51139999999998)
(4, 33.47949999999997)
(5, 34.94970000000001)
(6, 41.888919999999985)
(7, 45.38745)
(8, 42.3186)
(9, 42.74756000000002)
(10, 35.76049999999998)
(11, 24.174320000000023)
(12, 17.775969999999973)


In [28]:
# minimum teperature

minTemps = month_temperature_pair.reduceByKey(lambda a, b: min(a,b))
for element in minTemps.collect():
    print(element)

(1, -4.027699999999982)
(2, -1.2094000000000165)
(3, -4.883359999999982)
(4, 0.6752999999999929)
(5, 3.9842199999999934)
(6, 6.788819999999987)
(7, 9.896729999999991)
(8, 8.30419999999998)
(9, 5.084960000000024)
(10, 3.7046000000000276)
(11, -7.347140000000024)
(12, -5.877499999999998)


In [31]:
# average precipitation
month_precipitation_pair = parsed_data.map(lambda data: (data[1], data[3]))
month_precipitation_grouped = month_precipitation_pair.groupByKey();
for mp in month_precipitation_grouped.collect():
    count = 0;
    rain_sum = 0;
    for rain in mp[1]:
        count += 1
        rain_sum += rain
    print((mp[0], rain_sum/count))



(1, 0.29830751936047967)
(2, 0.6940859993403694)
(3, 0.41724514457883555)
(4, 0.2715082252451756)
(5, 0.08299456081907666)
(6, 0.05662295361164924)
(7, 0.05651621075891496)
(8, 0.14651450022367885)
(9, 0.13561652688419118)
(10, 0.2079679756004575)
(11, 0.4159643046370612)
(12, 1.2858842566152675)
